<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

ResNet50 = ResNet50(weights='imagenet')
features = model.predict(x)

```

Next you will need to remove the last layer from the ResNet model. Here, we loop over the layers to use the sequential API. There are easier ways to add and remove layers using the Keras functional API, but doing so introduces other complexities. 

```python
# Remote the Last Layer of ResNEt
ResNet50._layers.pop(0)

# Out New Model
model = Sequential()

# Add Pre-trained layers of Old Model to New Model
for layer in ResNet50.layers:
    model.add(layer)

# Turn off additional training of ResNet Layers for speed of assignment
for layer in model.layers:
    layer.trainable = False

# Add New Output Layer to Model
model.add(Dense(1, activation='sigmoid'))
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

In [1]:
### YOUR CODE HERE

import numpy as np
import pathlib
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model

In [2]:
### check this tutorial.  https://www.tensorflow.org/tutorials/load_data/images
url_montains = './data/mountain/*.jpg'
url_forest = './data/forest/*.jpg'

In [3]:
from skimage.io import imread_collection
#https://scikit-image.org/docs/dev/api/skimage.io.html#skimage.io.imread_collection
mountains = imread_collection(url_montains, conserve_memory=False)
forest = imread_collection(url_forest, conserve_memory=False)

In [4]:
len(mountains)

374

In [5]:
mountains[0].shape

(256, 256, 3)

In [6]:
mountains.concatenate().shape

(374, 256, 256, 3)

In [7]:
len(forest)

328

In [8]:
forest[0].shape

(256, 256, 3)

In [9]:
forest.concatenate()

array([[[[  3,   0,   0],
         [  5,   1,   0],
         [  6,   0,   0],
         ...,
         [  0,   0,  10],
         [ 10,  10,  20],
         [  0,   0,   9]],

        [[  3,   0,   0],
         [  7,   1,   1],
         [  8,   0,   0],
         ...,
         [ 11,  12,  17],
         [  3,   4,   9],
         [  0,   1,   6]],

        [[  5,   0,   0],
         [  9,   1,   0],
         [ 10,   0,   0],
         ...,
         [  0,   1,   3],
         [  0,   0,   2],
         [  4,   4,   6]],

        ...,

        [[  2,   0,   1],
         [ 13,  11,  12],
         [  6,   4,   5],
         ...,
         [ 14,   5,   0],
         [  8,   1,   0],
         [ 11,   3,   0]],

        [[  1,   2,   6],
         [  2,   3,   7],
         [  0,   0,   5],
         ...,
         [  4,   0,   0],
         [  5,   1,   0],
         [  4,   0,   0]],

        [[  0,   1,   6],
         [  0,   0,   8],
         [  2,   5,  12],
         ...,
         [  1,   1,   0],
        

In [10]:
#Concatenate all the image vertically
mountains = mountains.concatenate()
forest = forest.concatenate()

In [11]:
#Now concatenate both datasets
X = np.concatenate([forest, mountains]) 

In [12]:
X.shape

(702, 256, 256, 3)

In [13]:
#zeros are forests and ones mountains
y = np.concatenate([np.zeros(forest.shape[0]), np.ones(mountains.shape[0])])

In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [15]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((561, 256, 256, 3), (141, 256, 256, 3), (561,), (141,))

In [16]:
ResNet50 = ResNet50(input_shape=(256, 256, 3), weights='imagenet', include_top=False)
# include_top=False  not include the final pooling and fully connected layer in the original model

In [17]:
# To "freeze" a layer means to exclude it from training, i.e. its weights will never be updated. This is useful in the context of fine-tuning a model, or using fixed embeddings for a text input.
for layer in ResNet50.layers:
    layer.trainable = False

In [19]:
from tensorflow.keras.layers import Dropout

In [20]:
x = ResNet50.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(256, activation='relu')(x)

predictions = Dense(1, activation='sigmoid')(x)
model = Model(ResNet50.input, predictions)

In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [22]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5, validation_split=.2)

Train on 448 samples, validate on 113 samples
Epoch 1/5
448/448 [==============================] - 13s 29ms/sample - loss: 0.4548 - accuracy: 0.7857 - val_loss: 0.1345 - val_accuracy: 0.9558
Epoch 2/5
448/448 [==============================] - 4s 10ms/sample - loss: 0.1039 - accuracy: 0.9710 - val_loss: 0.0818 - val_accuracy: 0.9646
Epoch 3/5
448/448 [==============================] - 4s 10ms/sample - loss: 0.1183 - accuracy: 0.9576 - val_loss: 0.1866 - val_accuracy: 0.9381
Epoch 4/5
448/448 [==============================] - 4s 10ms/sample - loss: 0.0743 - accuracy: 0.9732 - val_loss: 0.0387 - val_accuracy: 0.9823
Epoch 5/5
448/448 [==============================] - 4s 10ms/sample - loss: 0.0667 - accuracy: 0.9710 - val_loss: 0.0878 - val_accuracy: 0.9646


In [23]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

141/1 - 2s - loss: 0.0735 - accuracy: 0.9645


# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language